# Predicting

In [1]:
import sys
sys.path.append("../")
from MapSci.papers import papers
from MapSci.research_space import research_space
from MapSci.entities import entities
import MapSci.agg as agg
import MapSci.predict as pr

TEXT

In [2]:
pp = papers("lattes")
maps = research_space("lattes", pp)
maps.compute(2000, 2014)

guevara(2000, 2014, 0.1)
chinazzi(2000, 2014, 0.1, 200)


In [3]:
presence = pp.presence(2000, 2014)
indices = maps.phi["guevara(2000, 2014, 0.1)"][1]
scientists = entities(presence, indices)

In [5]:
inst, st = agg.get_insts(scientists.set, "../dataset/lattes/pesquisadores.csv", sep=";sep;")

xi = agg.aggregate(presence[1], inst)
presence_inst = pp.presence(2000, 2014, x=xi)
institutions = entities(presence_inst, indices)

xs = agg.aggregate(presence[1], st)
presence_st = pp.presence(2000, 2014, x=xs)
states = entities(presence_st, indices)

### Prediction

In [9]:
guev = maps.phi["guevara(2000, 2014, 0.1)"][0]
chin = maps.phi["chinazzi(2000, 2014, 0.1, 200)"][0]

*evaluating the transition from an inactive to an active state*

In [22]:
# Laender
scientists.predict(9089204821424223, guev, 'inactive-active')[:10]

[(0.32805, 'museology'),
 (0.27583, 'signal processing'),
 (0.27155, 'theoretical computer science'),
 (0.26721, 'media technology'),
 (0.24403, 'logic'),
 (0.22847, 'computer graphics and computer-aided design'),
 (0.22656, 'human-computer interaction'),
 (0.17321, 'information systems and management'),
 (0.16858, 'control and systems engineering'),
 (0.15695, 'management information systems')]

In [32]:
# Ana
scientists.predict(2408991231058279, chin, 'inactive-active')[:10]

[(0.07867, 'information systems'),
 (0.06749, 'computer vision and pattern recognition'),
 (0.06358, 'computer science (miscellaneous)'),
 (0.06116, 'artificial intelligence'),
 (0.0604, 'theoretical computer science'),
 (0.05736, 'signal processing'),
 (0.05637, 'computer graphics and computer-aided design'),
 (0.05598, 'information systems and management'),
 (0.05582, 'computer science applications'),
 (0.05411, 'logic')]

*evaluating the transition from an nascent to an developed state*

In [18]:
# Universidade Federal de Minas Gerais
institutions.predict('universidade federal de minas gerais', guev, 'nascent-developed')[:10]

[(0.72025, 'family practice'),
 (0.71126, 'urology'),
 (0.61904, 'advanced and specialized nursing'),
 (0.60379, 'anatomy'),
 (0.5844, 'human factors and ergonomics'),
 (0.57649, 'logic'),
 (0.5606, 'critical care and intensive care medicine'),
 (0.55542, 'electrochemistry'),
 (0.53811, 'medical and surgical nursing'),
 (0.52879, 'radiological and ultrasound technology')]

*evaluating the transition from an intermediate to an developed state*

In [31]:
# Minas Gerais
states.predict('31', chin, 'intermediate-developed')[:10]

[(0.53724, 'nature and landscape conservation'),
 (0.53666, 'electrochemistry'),
 (0.52147, 'process chemistry and technology'),
 (0.51276, 'aquatic science'),
 (0.503, 'polymers and plastics'),
 (0.49625, 'surfaces and interfaces'),
 (0.4933, 'colloid and surface chemistry'),
 (0.47303, 'renewable energy, sustainability and the environment'),
 (0.46187, 'community and home care'),
 (0.46187, 'atmospheric science')]

### Accuracy